# "THE PRICE IS RIGHT" Capstone Project

This week - build a model that predicts how much something costs from a description, based on a scrape of Amazon data


A model that can estimate how much something costs, from its description.

# Order of play

DAY 1: Data Curation  
DAY 2: Data Pre-processing  
DAY 3: Evaluation, Baselines, Traditional ML  
DAY 4: Deep Learning and LLMs  
DAY 5: Fine-tuning a Frontier Model  

## DAY 4: Neural Networks and LLMs

Today we'll work from Traditional ML to Neural Networks to Large Language Models!!

In [1]:
import os
from dotenv import load_dotenv
from huggingface_hub import login
from pricer.evaluator import evaluate
from litellm import completion
from pricer.items import Item
import numpy as np
from tqdm.notebook import tqdm
import csv
from sklearn.feature_extraction.text import HashingVectorizer
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import CosineAnnealingLR

# GPU / CPU device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cpu


In [2]:
LITE_MODE = False

load_dotenv(override=True)
hf_token = os.getenv("HF_TOKEN")
login(hf_token, add_to_git_credential=True)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [3]:
username = "ed-donner"
dataset = f"{username}/items_lite" if LITE_MODE else f"{username}/items_full"

train, val, test = Item.from_hub(dataset)

print(f"Loaded {len(train):,} training items, {len(val):,} validation items, {len(test):,} test items")

Loaded 800,000 training items, 10,000 validation items, 10,000 test items


# Before we look at the Artificial Neural Networks

## There is a different kind of Neural Network we could consider

In [4]:
# Write the test set to a CSV

with open('human_in.csv', 'w', encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    for t in test[:100]:
        writer.writerow([t.summary, 0])

In [5]:
# Read it back in

human_predictions = []
with open('human_out.csv', 'r', encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        human_predictions.append(float(row[1]))

In [6]:
def human_pricer(item):
    idx = test.index(item)
    return human_predictions[idx]

In [7]:
human = human_pricer(test[0])
actual = test[0].price
print(f"Human predicted {human} for an item that actually costs {actual}")


Human predicted 120.0 for an item that actually costs 219.0


In [8]:
evaluate(human_pricer, test, size=100)

  0%|          | 0/100 [00:00<?, ?it/s]

$99 $184 $12 $15 $18 $10 $119 $135 $6 $270 $643 $329 $15 $26 $24 $18 $29 $25 $25 $53 $35 $126 $25 $127 $273 $398 $55 $6 $101 $51 $30 $5 $35 $9 $10 $419 $25 $11 $186 $33 $161 $51 $23 $155 $150 $4 $31 $18 $115 $82 $25 $111 $410 $75 $67 $34 $8 $10 $122 $28 $116 $17 $19 $60 $599 $60 $160 $355 $75 $34 $17 $2 $70 $76 $41 $9 $226 $5 $5 $4 $0 $7 $5 $74 $7 $10 $68 $74 $5 $3 $17 $45 $5 $16 $0 $153 $2 $122 $150 $355 

# And now - a vanilla Neural Network

During the remainder of this course we will get deeper into how Neural Networks work, and how to train a neural network.

This is just a sneak preview - let's make our own Neural Network, from scratch, using Pytorch.

Use this to get intuition; it's not important to know all about Neural networks at this point..

In [9]:
# Prepare our documents and prices

y = np.array([float(item.price) for item in train])
documents = [item.summary for item in train]

In [10]:
# Use the HashingVectorizer for a Bag of Words model
# Using binary=True with the CountVectorizer makes "one-hot vectors"

np.random.seed(42)
vectorizer = HashingVectorizer(n_features=5000, stop_words='english', binary=True)
X = vectorizer.fit_transform(documents)

In [11]:
# Define the neural network - here is Pytorch code to create a 8 layer neural network

class NeuralNetwork(nn.Module):
    def __init__(self, input_size):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(input_size, 128)
        self.layer2 = nn.Linear(128, 64)
        self.layer3 = nn.Linear(64, 64)
        self.layer4 = nn.Linear(64, 64)
        self.layer5 = nn.Linear(64, 64)
        self.layer6 = nn.Linear(64, 64)
        self.layer7 = nn.Linear(64, 64)
        self.layer8 = nn.Linear(64, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        output1 = self.relu(self.layer1(x))
        output2 = self.relu(self.layer2(output1))
        output3 = self.relu(self.layer3(output2))
        output4 = self.relu(self.layer4(output3))
        output5 = self.relu(self.layer5(output4))
        output6 = self.relu(self.layer6(output5))
        output7 = self.relu(self.layer7(output6))
        output8 = self.layer8(output7)
        return output8

In [12]:
class SparseCSRDataset(Dataset):
    def __init__(self, X_csr, y):
        self.X = X_csr          # scipy.sparse.csr_matrix
        self.y = y              # 1D numpy array

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        # Extract one row -> small dense vector
        row = self.X[idx].toarray().astype("float32")   # (1, n_features)
        x = torch.from_numpy(row).squeeze(0)            # (n_features,)
        target = torch.tensor(self.y[idx], dtype=torch.float32)
        return x, target



In [13]:
# Split indices instead of densifying X
indices = np.arange(X.shape[0])
train_idx, val_idx = train_test_split(indices, test_size=0.01, random_state=42)

X_train_csr = X[train_idx]
y_train = y[train_idx]
X_val_csr = X[val_idx]
y_val = y[val_idx]

train_dataset = SparseCSRDataset(X_train_csr, y_train)
val_dataset   = SparseCSRDataset(X_val_csr,   y_val)

trainloader = DataLoader(train_dataset, batch_size=256, shuffle=True)
valloader   = DataLoader(val_dataset,   batch_size=256, shuffle=False)

# Initialize the model
inputsize = X.shape[1]
model = NeuralNetwork(inputsize).to(device)   # move model to GPU/CPU

In [14]:
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Number of trainable parameters: {trainable_params:,}")

Number of trainable parameters: 669,249


In [15]:
lossfunction = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
EPOCHS = 2

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0

    for batchX, batchy in tqdm(trainloader):
        batchX = batchX.to(device)
        batchy = batchy.to(device).unsqueeze(1)  # (batch, 1)

        optimizer.zero_grad()
        outputs = model(batchX)
        loss = lossfunction(outputs, batchy)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * batchX.size(0)

    train_loss = running_loss / len(train_dataset)

    # Validation
    model.eval()
    val_losses = []
    with torch.no_grad():
        for Xb, yb in valloader:
            Xb = Xb.to(device)
            yb = yb.to(device).unsqueeze(1)
            vout = model(Xb)
            valloss = lossfunction(vout, yb)
            val_losses.append(valloss.item())

    print(
        f"Epoch {epoch + 1}/{EPOCHS}, "
        f"Train Loss {train_loss:.3f}, "
        f"Val Loss {np.mean(val_losses):.3f}"
    )


  0%|          | 0/3094 [00:00<?, ?it/s]

Epoch 1/2, Train Loss 15190.022, Val Loss 13296.436


  0%|          | 0/3094 [00:00<?, ?it/s]

Epoch 2/2, Train Loss 11924.909, Val Loss 11556.320


In [16]:
def neuralnetwork(item):
    model.eval()
    with torch.no_grad():
        # vectorizer.transform expects an iterable of strings
        vec = vectorizer.transform([item.summary])   # 1 x 5000 sparse
        row = vec.toarray().astype("float32")        # small dense array
        x = torch.from_numpy(row).to(device)         # move to GPU/CPU
        result = model(x)[0, 0].item()
        return max(0, result)


In [17]:
evaluate(neuralnetwork, test)

  0%|          | 0/200 [00:00<?, ?it/s]

$167 $72 $23 $103 $57 $126 $72 $4 $42 $116 $182 $230 $61 $110 $39 $13 $3 $8 $22 $43 $5 $48 $67 $0 $208 $95 $198 $45 $146 $41 $116 $145 $55 $4 $53 $437 $16 $74 $104 $88 $159 $21 $13 $10 $38 $50 $41 $32 $58 $35 $29 $60 $176 $18 $63 $133 $29 $149 $9 $51 $66 $15 $22 $26 $323 $67 $56 $337 $10 $182 $12 $27 $63 $128 $13 $7 $92 $17 $27 $70 $63 $70 $62 $54 $15 $101 $124 $78 $16 $152 $15 $49 $2 $1 $36 $29 $34 $45 $32 $215 $4 $23 $19 $29 $9 $36 $55 $251 $9 $49 $28 $87 $190 $20 $12 $144 $31 $82 $29 $149 $38 $224 $118 $35 $4 $37 $28 $109 $46 $24 $40 $51 $77 $28 $33 $20 $72 $14 $17 $46 $18 $110 $35 $92 $89 $44 $38 $300 $52 $0 $18 $130 $9 $32 $29 $116 $37 $3 $16 $17 $3 $5 $4 $18 $335 $28 $12 $31 $12 $66 $36 $19 $273 $39 $8 $24 $62 $8 $41 $122 $88 $18 $128 $107 $13 $55 $79 $70 $25 $30 $44 $3 $0 $101 $4 $19 $80 $22 $9 $1 

# And now - to the frontier!

Let's see how Frontier models do out of the box; no training, just inference based on their world knowledge.

Tomorrow we will do some training.

In [ ]:
def messages_for(item):
    message = f"Estimate the price of this product. Respond with the price, no explanation\n\n{item.summary}"
    return [{"role": "user", "content": message}]

In [ ]:
print(test[0].summary)

In [ ]:
messages_for(test[0])

In [ ]:
# The function for gpt-4.1-nano

def gpt_4__1_nano(item):
    response = completion(model="openai/gpt-4.1-nano", messages=messages_for(item), api_key=userdata.get('OPENAI_API_KEY'))
    return response.choices[0].message.content

In [ ]:
gpt_4__1_nano(test[0])

In [ ]:
test[0].price

In [ ]:
evaluate(gpt_4__1_nano, test)

In [ ]:
def claude_opus_4_5(item):
    response = completion(model="anthropic/claude-opus-4-5", messages=messages_for(item))
    return response.choices[0].message.content

In [ ]:
evaluate(claude_opus_4_5, test)

In [ ]:
def gemini_3_pro_preview(item):
    response = completion(model="gemini/gemini-3-pro-preview", messages=messages_for(item), reasoning_effort='low')
    return response.choices[0].message.content

In [ ]:
evaluate(gemini_3_pro_preview, test, size=50, workers=2)

In [ ]:
def gemini_2__5_flash_lite(item):
    response = completion(model="gemini/gemini-2.5-flash-lite", messages=messages_for(item))
    return response.choices[0].message.content

In [ ]:
evaluate(gemini_2__5_flash_lite, test)

In [ ]:

def grok_4__1_fast(item):
    response = completion(model="xai/grok-4-1-fast-non-reasoning", messages=messages_for(item), seed=42)
    return response.choices[0].message.content

In [ ]:
evaluate(grok_4__1_fast, test)

In [ ]:
# The function for gpt-5.1

def gpt_5__1(item):
    response = completion(model="gpt-5.1", messages=messages_for(item), reasoning_effort='high', seed=42)
    return response.choices[0].message.content


In [ ]:
evaluate(gpt_5__1, test)